# History and Data Analysis Collaboration System

This notebook implements a multi-agent collaboration system that combines historical research with data analysis to answer complex historical questions. It leverages the power of large language models to simulate specialized agents working together to provide comprehensive answers.

## Key Components
1. **Agent Class**
2. **History Research Agent**
3. **Data Analysis Agent**
4. **History Data Collaboration System**

In [1]:
import os
import time

from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from typing import List, Dict
from dotenv import load_dotenv

load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv('GOOGLE_API_KEY')

/Users/tetro/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [ ]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)